## Librerias a utilizar

In [47]:
#!pip install pyarrow

In [48]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import re
import time
from datetime import date
import pyarrow

# Manejo Web, paginas y webScrapping
# ==============================================================================
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs


# Gráficos
# ==============================================================================
from matplotlib import pyplot as plt


# Configuración warnings
# ==============================================================================\n",
import warnings
warnings.filterwarnings('ignore')

## Funciones 

In [49]:
def ExtraerLink(linkPage,patron):
    lista = []
    for tag in linkPage:
        valor = tag.get('href')
        if(str(valor).find(patron) != -1):
            lista.append(valor)
    df = pd.DataFrame (lista, columns = ['url'])
    df = df.drop_duplicates()
    return df
def leerUrl(pagina):    
    soup = bs(urllib.request.urlopen(pagina).read().decode())
    #print(str(soup) )
    time.sleep(5)
  
    return  soup



## Extraccion de Oferta Laborales Linkedin

In [50]:
# Instantiate the webdriver with the executable location of MS Edge
# Provide the full location of the path to recognise correctly
PATH = 'msedgedriver.exe'
options = webdriver.EdgeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_argument('disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument('--headless')
driver = webdriver.Edge(PATH, options= options)

In [51]:
# Browser will get navigated to the gi((ven URL
driver.get('https://www.linkedin.com/jobs/search/?keywords=Data%20Scientist&location=Chile&locationId=&geoId=104621616&f_TPR=r86400&position=1&pageNum=0')
time.sleep(3)

## Preprocesamiento de Pagina de Empleo LinkedIn
Se crea un Dataframe con los link de las paginas de Oferta

In [52]:
linkedin_soup = bs(driver.page_source.encode("utf-8"), "html")
#print(linkedin_soup)
patron = '/jobs/view/'
df = ExtraerLink(linkedin_soup('a'),patron)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     27 non-null     object
dtypes: object(1)
memory usage: 432.0+ bytes


## Creacion de DF con resultado de paginas


In [53]:
columns = ["url", "contenido"]
dffinal = pd.DataFrame(columns=columns)

for i in range(len(df)-1):
    link = str(df.iloc[i]['url'])

    #name = get_name(driver,link)   
    #print(name)
    linkedin_soup1 = leerUrl(link.split('?')[0]) 
    #print(linkedin_soup1)
    if str(linkedin_soup1) == "Not Found":
        break
 
    parametro = 'show-more-less-html__markup'
    links_divs = linkedin_soup1.findAll("div", {"class": parametro})
    links_divs = str(links_divs)
    dffinal = dffinal.append(
                {
                    "url": link,
                    "contenido": links_divs,
                },
                ignore_index=True,
            )

## Guardar DataFrame con valores Crudos, para preprocesamiento de datos.

In [54]:
# Traza
#with pd.option_context('display.max_colwidth', None):
#    display(dffinal)

dffinal.to_parquet("output/df_"+date.today().strftime("%d%m%Y")+'.parquet', engine='pyarrow')

In [55]:
dffinal.contenido[2]

'[<div class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5">\n<p>A globally recognized biotechnology startup who develop hyperfood (therapeutically functional foods made by artificial intelligence).</p><p><br/></p><p>Joining their global machine learning space, you will be building next-generation human food  using Machine Learning/AI to help expand our business footprint, you could be doing ground-breaking work that brings our iconic, global brand into the future. If you love to work help human with healthy food with Machine Learning/AI using the latest technology, we’ll support you with an open and learning environment to grow your career.</p><p><br/></p><p><u>Requirements:</u></p><p><br/></p><ul><li>Computer science or related degree</li><li>At least 2+ years of experience in ML development, ideally Python as a programming language and Pytorch</li><li>Highly collaborative individuals who want to work in flat structured teams and a desire to own projects</li

In [46]:
str(df.iloc[0]['url'])

'https://cl.linkedin.com/jobs/view/cient%C3%ADfico-de-datos-at-ibm-3252800220?refId=MOw5HN9ZLqp%2FZikoNyR0lA%3D%3D&trackingId=3B9h9aVJ6JQlgXLQREimPw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'